### Generate simulated HCI images for testing PACO and other algorithms
Evert Nasedkin

In [1]:
import numpy as np
import os
from astropy.io import fits
import matplotlib.pyplot as plt

import hcipy as hci

## Simulated coronographic images using hcipy

In [2]:
# Telescope and Wavelength parameters
D_tel = 6.5 # meter
wavelength = 2.5e-6 # meter

# Generate pupil and focal plane grids, and the wavefront propogator
pupil_grid = hci.make_pupil_grid(1024,D_tel)
focal_grid = hci.make_focal_grid(pupil_grid, 4, 8, wavelength=wavelength)
prop = hci.FraunhoferPropagator(pupil_grid, focal_grid)

# Build the pupil aperture

# Circular Aperture
#aperture = hci.circular_aperture(D_tel)
#aperture = hci.evaluate_supersampled(aperture, pupil_grid, 8)

# Magellan Aperture
aperture = hci.make_magellan_aperture()

# Adaptive Optics
F_mla = 0.03
N_mla = 8
D_mla = 1.0 / N_mla
#x = np.arange(-1,1,D_mla)
#mla_grid = hci.CartesianGrid(SeparatedCoords((x,x)))
#mla_shape = hci.rectangular_aperture(D_mla)
#microlens_array = hci.MicroLensArray(pupil_grid, mla_grid, F_mla * D_mla, mla_shape)
#sh_prop = hci.FresnelPropagator(pupil_grid, F_mla * D_mla)


In [ ]:
im_stack = []
psfs = []
nFrames = 150
tInt = 1.5
# Generate the coronagraph
coro = hci.VortexCoronagraph(pupil_grid, charge=2, levels=8)
lyot_stop = hci.Apodizer(hci.circular_aperture(0.99*D_tel)(pupil_grid))

for i in range(nFrames):
    # Generate a wavefront
    #wf = hci.Wavefront(aperture,wavelength)
    wf = hci.Wavefront(aperture(pupil_grid),wavelength)
    wf.total_power = 100000

    # Atmospheric Distortion Layers
    fried_parameter = 5.5 # meter
    outer_scale = 20 # meter
    velocity = 1 # meter/sec

    # Single Layer
    Cn_squared = hci.Cn_squared_from_fried_parameter(fried_parameter, wavelength)
    layer = hci.InfiniteAtmosphericLayer(pupil_grid, Cn_squared, outer_scale, velocity)
    layer.t = i * tInt
    wf2 = layer(wf)

    # Multi Layer
    #spectral_noise_factory = hci.SpectralNoiseFactoryFFT(kolmogorov_psd, pupil_grid, 8)
    #turbulence_layers = hci.make_standard_multilayer_atmosphere(fried_parameter, wavelength=wavelength)
    #atmospheric_model = hci.AtmosphericModel(spectral_noise_factory, turbulence_layers)
    #wf2 = atmospheric_model(wf)
    #sci_img = prop(wf2).intensity
    #wf3 = microlens_array(wf2)
    #wfs_img = sh_prop(wf3).intensity

    # Generate surface aberration
    aberration = hci.SurfaceAberration(pupil_grid, 0.25*wavelength, D_tel)
    ab_wf = aberration(wf2)
    
    # Lyot Plane
    lyot_wf = coro(ab_wf)

    # Add a Lyot Stop
    lyot_img = prop(lyot_stop(lyot_wf))
    img = lyot_img
    img_ref = prop(wf)
    # Build a Detector
    flat_field = 0.1
    dark = 10
    detector = hci.NoisyDetector(focal_grid, dark_current_rate=dark, flat_field=flat_field)
    detector.integrate(lyot_img, tInt)
    image = detector.read_out()
    im_stack.append(image)
    img = prop(aberration(wf))
    detector.integrate(img, 1.5)
    psf = detector.read_out()
    psfs.append(psf)

/home/evert/.local/lib/python3.7/site-packages/hcipy/fourier/fast_fourier_transform.py:70: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  f[self.cutout_input] = (field.ravel() * self.weights * self.shift_output).reshape(self.shape_in)
/home/evert/.local/lib/python3.7/site-packages/hcipy/fourier/fast_fourier_transform.py:72: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  res = res[self.cutout_output].ravel() * self.shift_input
/home/evert/.local/lib/python3.7/site-packages/hcipy/fourier/fast_fourier_transform.py:81: FutureWarning: Using a no

In [ ]:
# Show the wavefront
#hci.imshow_field(np.log10(img.intensity / img_ref.intensity.max()), vmin=-4)
plt.figure(figsize = (6,5))
hci.imshow_field(im_stack[0], vmax=im_stack[0].max(), vmin=0)

plt.colorbar()
plt.show()

In [ ]:
psfs = np.asarray(psfs).reshape((nFrames,64,64))
print(psfs.shape)
plt.figure(figsize = (6,5))
plt.imshow(np.median(psfs,axis = 0)/np.sum(np.median(psfs,axis = 0)))
plt.colorbar()
plt.show()

In [ ]:
hdu = fits.PrimaryHDU(np.asarray(im_stack).reshape((nFrames,64,64)))
hdu.writeto("testData/SimImages.fits")

In [ ]:
hdu2 = fits.PrimaryHDU(np.median(psfs,axis = 0)/np.sum(np.median(psfs,axis = 0)))
hdu2.writeto("testData/SimImagesPSF.fits")

## Exoplanet injection

In [357]:
from pynpoint.processing.fluxposition import FakePlanetModule
from pynpoint import Pypeline, \
                     Hdf5ReadingModule, \
                     FitsReadingModule, \
                     AngleCalculationModule, \
                     ParangReadingModule
from paco.util.util import *
from astropy.io import fits

In [374]:
working_dir = "/home/evert/Documents/PACO/"
input_dir = working_dir + "testData/vip_datasets/"
output_dir = working_dir + "testData/"
data_filename = "bpic_data/naco_betapic_cube.fits"
psf_filename = "bpic_psf/naco_betapic_psf.fits"
parang_filename = "naco_betapic_pa.dat"
out_filename = "naco_betapic_injected.fits"

#xx, yy = np.meshgrid(np.arange(-50, 51),np.arange(-50, 51))
#s = gaussian2d(xx,yy,20, 2)
#s = np.array([s/np.sum(s)])
#hdu = fits.PrimaryHDU(s)
#hdu.writeto(input_dir + "gausspsf/gausspsf.fits")

In [375]:
pipeline = Pypeline(working_place_in = working_dir,
                    input_place_in = input_dir,
                    output_place_in = output_dir)

Initiating PynPoint v0.8.0... [DONE]


In [376]:
module = FitsReadingModule(name_in = "read1",
                          input_dir = input_dir + "bpic_data/",
                          image_tag = 'science0',
                          overwrite = True)
pipeline.add_module(module)

In [377]:
module = FitsReadingModule(name_in = psf_filename,
                          input_dir = input_dir + "gausspsf/",
                          image_tag = 'psf',
                          overwrite = True)
pipeline.add_module(module)

In [378]:
#module = AngleCalculationModule(name_in='angle',
#                                data_tag='last',
#                                instrument='NACO')

#pipeline.add_module(module)

# OR

module = ParangReadingModule(name_in = "parang_reading",
                             data_tag = 'science0',
                             file_name = parang_filename,
                             input_dir = input_dir)
pipeline.add_module(module)

In [379]:
# Repeat for each planet to be injected
#posn = (1.4,30.0)  # (sep angle [as], posn angle [deg])
#magnitude = 2.0    # Magnitude relative to star

#module = FakePlanetModule(name_in = 'fake_planet',
#                          image_in_tag = 'science',
#                          psf_in_tag = 'psf',
#                          image_out_tag = 'injected',
#                          position = posn,
#                          magnitude = magnitude)
#pipeline.add_module(module)

############
posn_list = [(0.1,30),(0.3,30),(0.5,30),(0.7,30),(0.2,90),(0.5,90),(0.8,90),(0.3,180),(0.5,180),(0.5,180)]
scale =     [1, 0.8, 0.6, 0.4,                     0.7, 0.5, 0.3,              0.3, 0.2, 0.1]
count = 0
for i in range(len(posn_list)):
    module = FakePlanetModule(name_in = 'fake_planet_'+ str(i),
                          image_in_tag = 'science'+str(i),
                          psf_in_tag = 'psf',
                          image_out_tag = 'science'+str(i+1),
                          position = posn_list[i],
                          magnitude = 0.0,
                          psf_scaling = scale[i])
    count = i+1
    pipeline.add_module(module)

In [380]:
from pynpoint import FitsWritingModule
module = FitsWritingModule(name_in='write',
                           file_name=out_filename,
                           output_dir=output_dir,
                           data_tag='science'+str(count),
                           data_range=None)

pipeline.add_module(module)

In [381]:
pipeline.run()

Validating Pypeline... [DONE]
Running FitsReadingModule... [DONE]                      
Running FitsReadingModule... [DONE]                      
Running ParangReadingModule... [DONE]
Running FakePlanetModule... [DONE]                      


/home/evert/.local/lib/python3.7/site-packages/pynpoint/util/attributes.py:75: UserWarning: Static attribute INSTRUMENT (=NACO) not found in the FITS header.
  warnings.warn(f'Static attribute {attr} (={fitskey}) not found in the FITS '
/home/evert/.local/lib/python3.7/site-packages/pynpoint/util/attributes.py:75: UserWarning: Static attribute DIT (=ESO DET DIT) not found in the FITS header.
  warnings.warn(f'Static attribute {attr} (={fitskey}) not found in the FITS '
/home/evert/.local/lib/python3.7/site-packages/pynpoint/util/attributes.py:75: UserWarning: Static attribute LATITUDE (=ESO TEL GEOLAT) not found in the FITS header.
  warnings.warn(f'Static attribute {attr} (={fitskey}) not found in the FITS '
/home/evert/.local/lib/python3.7/site-packages/pynpoint/util/attributes.py:75: UserWarning: Static attribute LONGITUDE (=ESO TEL GEOLON) not found in the FITS header.
  warnings.warn(f'Static attribute {attr} (={fitskey}) not found in the FITS '
/home/evert/.local/lib/python3.7/si

Running FakePlanetModule... [DONE]
Running FakePlanetModule... [DONE]                      
Running FakePlanetModule... [DONE]                      
Running FakePlanetModule... [DONE]                      
Running FakePlanetModule... [DONE]                      
Running FakePlanetModule... [DONE]                      
Running FakePlanetModule... [DONE]                      
Running FakePlanetModule... [DONE]                      
Running FakePlanetModule... [DONE]                      
Running FitsWritingModule... [DONE]
